In [1]:
import numpy as np

## Задача 1

Даны значения зарплат из выборки выпускников:

100, 80, 75, 77, 89, 33, 45, 25, 65, 17, 30, 230, 24, 57, 55, 70, 75, 65, 84, 90, 150

Используя только встроенные питоновские функции и структуры данных (т.е. без библиотек numpy, pandas и др.) посчитать среднее арифметическое, выборочную дисперсию, среднее квадратичное отклонение для данной выборки.

Можно затем посчитать те же значения с использованием библиотек, чтобы проверить себя.

======================================================================================================================

In [2]:
def my_mean(input: (list, tuple, set), ddof=0):
    if isinstance(input, (list, tuple, set)):
        my_sum = 0
        for i in input:
            my_sum += i
        return my_sum / (len(input) - ddof)
    else:
        print("This is inappropriate input")

In [14]:
a = [100, 80, 75, 77, 89, 33, 45, 25, 65, 17, 30, 230, 24, 57, 55, 70, 75, 65, 84, 90, 150]

In [15]:
my_mean(a)

73.14285714285714

In [16]:
np.array(a).mean()

73.14285714285714

In [17]:
def my_var(input: (list, tuple, set), ddof=0):
    if isinstance(input, (list, tuple, set)):
        sum_of_squares = [(i - my_mean(input, ddof=0)) ** 2 for i in input]
        return my_mean(sum_of_squares, ddof)
    else:
        print("This is inappropriate input")

In [18]:
my_var(a, ddof=1)

2241.8285714285716

In [19]:
np.array(a).var(ddof=1)

2241.828571428571

In [20]:
def my_std(input: (list, tuple, set), ddof=0):
    if isinstance(input, (list, tuple, set)):
        return my_var(input, ddof) ** 0.5
    else:
        print("This is inappropriate input")

In [21]:
my_std(a, ddof=1)

47.34795213553139

In [22]:
np.array(a).std(ddof=1)

47.347952135531386

## Задача 2

Для выборки из задачи 1 найти (также без использования библиотек):
1. медиану, первый и третий квартили, интерквартильное расстояние,
2. выборсы в выборке (используя для этого метод как при построении "усов" из boxplot).

Возможные неоднозначности в вычислении квантилей можно разрешать любым способом.

================================================================================================================

In [26]:
def my_quantile(input: (list, tuple, set), quantile=0.5):
    """
    без интерполяции, выбор квантили только из предоставленного массива
    """
    if isinstance(input, (list, tuple, set)):
        if 0 < quantile <= 1:
            if len(input) * quantile == 0:
                return 0
            else:
                input = sorted(input)
                index_of_element = int(len(input) * quantile)
                return input[index_of_element]
        else:
            prine("Quantile must be in range (0, 1]")
    else:
        print("This is inappropriate input")

In [27]:
my_quantile(a)

70

In [29]:
np.quantile(np.array(a), q=0.5)

70.0

Собственно реализация вроде рабочая, хотя ответ моет отличаться, потому что мой алгоритм выдает только значения, которые находятся непосредственно в выборке. Найдем далее межквартильный размах

In [32]:
my_quantile(a, 0.75) - my_quantile(a, 0.25)

39

In [33]:
np.quantile(np.array(a), q=0.75) - np.quantile(np.array(a), q=0.25)

39.0

С нампаем сходиться.....уже хорошо))  Выбросы будем оценивать как первая квантиль минус полтора межквартильных размаха и третья квантиль плюс полтора межквартильных размаха....и все что не входит в этот интервал будем признавать выбросом

In [34]:
a_emissions = []
for el in a:
    lower_border = my_quantile(a, 0.25) - (my_quantile(a, 0.75) - my_quantile(a, 0.25)) * 1.5
    upper_boder = my_quantile(a, 0.27) + (my_quantile(a, 0.75) - my_quantile(a, 0.25)) * 1.5
    if not (lower_border <= el <= upper_boder):
        a_emissions.append(el)

In [35]:
a_emissions

[230, 150]

Ну собственно выбросы получились только верхние....потому что нижняя граница по расчету скорее всего в отрицательной зоне оказалась

In [36]:
my_quantile(a, 0.25) - (my_quantile(a, 0.75) - my_quantile(a, 0.25)) * 1.5

-13.5

======================================================================================================================

## Задача 3

В университет на факультет A поступило 100 человек, на факультет B - 90 человек и на факультет C - 45 человек. Вероятность того, что студент с факультета A сдаст первую сессию, равна 0.6. Для студента с факультета B эта вероятность равна 0.7, а для студента с факультета C - 0.8. Случайным образом выбирается студент, сдавший первую сессию. Какое событие наиболее вероятно:
1. студент учится на факультете A,
2. студент учится на факультете B,
3. студент учится на факультете C?

================================================================================================================

В данной задаче речь идет о наступлении события А при наступлении события В. Для нас событием А является поступление студента на однин из трех факультетов. А событием В то, что сдал ли он первую сессию или нет. В числителе нам необходимо учитывать вероятность успешной сдачи сессии студента с одного из трех факультетов умноженную на вероятность поступить на соответсвующий факультет. А в знаменателе общую вероятность сдать экзамены.  Хотя все это будет выражено в человеках, но на выходе будет вероятность)).  Итак общее количество человек, которые сдадут экзамены будет равно:

In [37]:
b_total = 100 * 0.6 + 90 * 0.7 + 45 * 0.8

In [39]:
print(f"Вероятность того что случайно выбранный студент, сдавший первую сессию учиться на факультете А равна \
      '{(100/(100+90+45) * 0.6) / b_total:%}'")
print(f"Вероятность того что случайно выбранный студент, сдавший первую сессию учиться на факультете B равна \
      '{(95/(100+90+45) * 0.7) / b_total:%}'")
print(f"Вероятность того что случайно выбранный студент, сдавший первую сессию учиться на факультете А равна \
      '{(45/(100+90+45) * 0.8) / b_total:%}'")

Вероятность того что случайно выбранный студент, сдавший первую сессию учиться на факультете А равна       '0.160578%'
Вероятность того что случайно выбранный студент, сдавший первую сессию учиться на факультете B равна       '0.177974%'
Вероятность того что случайно выбранный студент, сдавший первую сессию учиться на факультете А равна       '0.096347%'
